In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

import os
os.chdir('/home/cat/code/manifolds/')

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange, tqdm

from scipy.io import loadmat

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial import ConvexHull, convex_hull_plot_2d

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds


In [5]:
############################################
################ LOAD DATA #################
############################################
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
session = '20210519'

binarization_method='upphase'

#
c = calcium.Calcium()
c.root_dir = root_dir
c.animal_id = animal_id
c.session = session
c.load_suite2p()

# 
c.load_binarization()
if binarization_method=='onphase':
    traces = c.F_onphase_bin
elif binarization_method=='upphase':
    traces = c.F_upphase_bin
else:
    print ("METHOD NOT FOUND")


c.load_footprints()
    
print ("binarized data: ", traces.shape)

w = wheel.Wheel()
w.root_dir = os.path.join(root_dir,
                          animal_id,
                          session,
                          'TRD-2P')
w.animal_id = animal_id
w.session = session
w.load_track()
w.compute_velocity()


# 
ensembles = scipy.io.loadmat('/media/cat/4TB/donato/DON-006084/20210519/suite2p/plane0/ensembles/Upphase_sum_binned_ICA-ASSEMBLIES.mat')
print (ensembles.keys())





(531, 55740)
binarized data:  (531, 55740)
dict_keys(['__header__', '__version__', '__globals__', 'cs_assembly_vectors', 'cs_assemblies', 'ks_alpha', 'mp_assembly_vectors', 'mp_assemblies'])


In [8]:
###########################################################
###########################################################
###########################################################
states = ensembles['cs_assemblies']
#states = ensembles['assemblies']
print ("# ensembles: ", states.shape)

# 
# enumerate post-cleaned cells
all_cells = np.arange(c.F.shape[0])

# find which cells are good 
idx2 = np.where(c.iscell[:,0])[0]
print (idx2.shape)
fig=plt.figure(figsize=(20,20))
for k in range(len(states)):
    ax=plt.subplot(5,5,k+1)
    #ax=plt.subplot(3,3,k+1)
    idx_state = states[k][0].squeeze()
       
    # find which states
    idx_good = []
    for id_ in idx_state:
        if id_ in idx2:
            idx_good.append(id_)
           
    #print (idx_good)

    #
    final_ids = []
    for p in range(len(idx_good)):
        temp = idx_good[p]
        idx4 = np.where(idx2==temp)[0]
        final_ids.append(idx4)
        p#rint (temp, idx4)
    final_ids = np.hstack(final_ids)
    
    # 
    for id2 in final_ids:
        contour = c.get_footprint_contour(id2)
        plt.plot(contour[:,1],
                contour[:,0],
                c='black')
    plt.xlim(0,512)
    plt.ylim(0,512)
    plt.xticks([])
    plt.yticks([])
    plt.ylabel(str(k) + ", "+str(len(final_ids)))
    
if False:
    plt.savefig('/home/cat/ica_ensembles.svg')
    plt.close()
else:
    plt.show()

# ensembles:  (25, 1)
(531,)
